In [56]:
import polars as pl
import matplotlib.pyplot as plt

from common.constants.column_types import CPZP_SCHEMA, OZP_SCHEMA
from common.constants.column_names import SHARED_COLUMNS, OZP_COLUMNS

pl.Config.set_tbl_rows(100)
pl.Config.set_tbl_cols(100)

polars.config.Config

In [ ]:
FILE_PATH = "./DATACON_data/OZP_preskladane.csv"

In [ ]:
def read_preskladane_data(file_path: str, schema: pl.Schema) -> pl.DataFrame:
    return pl.read_csv(
        file_path,
        null_values=["NA", ""],
        schema=schema,
    )

In [19]:
ozp_df = read_preskladane_data(FILE_PATH, OZP_SCHEMA)

print("OZP DATA: ", ozp_df.schema)

OZP DATA:  Schema({'Id_pojistence': String, 'Pohlavi': Enum(categories=['M', 'F']), 'Rok_narozeni': Int64, 'Posledni_zahajeni_pojisteni': Date, 'Posledni_ukonceni_pojisteni': Date, 'Datum_umrti': Date, 'Typ_udalosti': Enum(categories=['předpis', 'vakcinace']), 'Detail_udalosti': Int64, 'Nazev': String, 'Pocet_baleni': Float64, 'Datum_udalosti': Date, 'léková_forma_zkr': String, 'ATC_skupina': String, 'síla': String, 'doplněk_názvu': String, 'léková_forma': String, 'léčivé_látky': String, 'Equiv_sloucenina': Enum(categories=['TRIAMCINOLON', 'FLUDROKORTISON', 'METHYLPREDNISOLON', 'DEXAMETHASON', 'PREDNISON', 'BETAMETHASON', 'HYDROKORTISON']), 'Prednison_equiv': Float64, 'Pocet_v_baleni': Float64, 'pocet_vakcinaci': Int64, 'ockovany': Enum(categories=['1', '0']), 'pocet_predpisu': Int64})


In [20]:
ozp_alive_df = ozp_df.filter(pl.col(SHARED_COLUMNS.DATUM_UMRTI).is_null())

print(ozp_alive_df)

shape: (3_421_491, 23)
┌────────────┬─────────┬────────────┬───────────┬───┬───────────┬───────────┬──────────┬───────────┐
│ Id_pojiste ┆ Pohlavi ┆ Rok_naroze ┆ Posledni_ ┆ … ┆ Pocet_v_b ┆ pocet_vak ┆ ockovany ┆ pocet_pre │
│ nce        ┆ ---     ┆ ni         ┆ zahajeni_ ┆   ┆ aleni     ┆ cinaci    ┆ ---      ┆ dpisu     │
│ ---        ┆ enum    ┆ ---        ┆ pojisteni ┆   ┆ ---       ┆ ---       ┆ enum     ┆ ---       │
│ str        ┆         ┆ i64        ┆ ---       ┆   ┆ f64       ┆ i64       ┆          ┆ i64       │
│            ┆         ┆            ┆ date      ┆   ┆           ┆           ┆          ┆           │
╞════════════╪═════════╪════════════╪═══════════╪═══╪═══════════╪═══════════╪══════════╪═══════════╡
│ ANPOI00087 ┆ F       ┆ 1962       ┆ 2016-07-3 ┆ … ┆ null      ┆ 0         ┆ 0        ┆ 0         │
│ 376        ┆         ┆            ┆ 1         ┆   ┆           ┆           ┆          ┆           │
│ ANPOI00148 ┆ F       ┆ 1979       ┆ 1995-01-0 ┆ … ┆ null      ┆ 0 

In [51]:
def get_vekova_kohorta(vek):
    if vek >= 80:
        return "80+"
    elif vek >= 70:
        return "70-79"
    elif vek >= 65:
        return "65-69"
    elif vek >= 60:
        return "60-64"
    elif vek >= 50:
        return "50-59"
    elif vek >= 30:
        return "30-49"
    elif vek >= 12:
        return "12-29"
    else:
        return "<12"


ocko_df = (
    ozp_alive_df.filter(pl.col("Typ_udalosti") == "vakcinace")
    .sort(["Id_pojistence", "Datum_udalosti"])
    .with_columns(pl.arange(0, pl.len()).over("Id_pojistence").alias("Cislo_davky") + 1)
    .with_columns(
        (pl.col("Datum_udalosti").dt.year() - pl.col("Rok_narozeni")).alias(
            "Vek_v_dobe_vakcinace"
        )
    )
    .with_columns(
        pl.col("Vek_v_dobe_vakcinace")
        .map_elements(get_vekova_kohorta, return_dtype=pl.Utf8)
        .alias("Vekova_kohorta")
    )
    .select(
        [
            "Id_pojistence",
            pl.col("Datum_udalosti").alias("Datum_vakcinace"),
            "Cislo_davky",
            "Vekova_kohorta",
        ]
    )
)


predpis_df = (
    ozp_alive_df.filter(pl.col("Typ_udalosti") == "předpis")
    .sort(["Id_pojistence", "Datum_udalosti"])
    .with_columns(
        pl.arange(0, pl.len()).over("Id_pojistence").alias("cislo_predpisu") + 1
    )
    .with_columns(
        (pl.col("Datum_udalosti").dt.year() - pl.col("Rok_narozeni")).alias(
            "Vek_v_dobe_predpisu"
        )
    )
    .with_columns(
        pl.col("Vek_v_dobe_predpisu")
        .map_elements(get_vekova_kohorta, return_dtype=pl.Utf8)
        .alias("Vekova_kohorta")
    )
    .select(
        [
            "Id_pojistence",
            pl.col("Datum_udalosti").alias("Datum_predpisu"),
            "cislo_predpisu",
            "Vekova_kohorta",
        ]
    )
)

print(ocko_df)
print(predpis_df)

predpis_with_ocko_df = (
    predpis_df.join_asof(
        ocko_df,
        left_on="Datum_predpisu",
        right_on="Datum_vakcinace",
        by="Id_pojistence",
        strategy="backward",
    )
    .with_columns(
        [pl.col("Cislo_davky").fill_null(0).alias("Pocet_davek_v_dobe_predpisu")]
    )
    .select(
        [
            "Id_pojistence",
            "Datum_predpisu",
            "cislo_predpisu",
            "Vekova_kohorta",
            "Pocet_davek_v_dobe_predpisu",
        ]
    )
)

print(predpis_with_ocko_df)

shape: (1_311_187, 4)
┌───────────────┬─────────────────┬─────────────┬────────────────┐
│ Id_pojistence ┆ Datum_vakcinace ┆ Cislo_davky ┆ Vekova_kohorta │
│ ---           ┆ ---             ┆ ---         ┆ ---            │
│ str           ┆ date            ┆ i64         ┆ str            │
╞═══════════════╪═════════════════╪═════════════╪════════════════╡
│ ANPOJ00000001 ┆ 2021-07-26      ┆ 1           ┆ 12-29          │
│ ANPOJ00000001 ┆ 2021-09-06      ┆ 2           ┆ 12-29          │
│ ANPOJ00000001 ┆ 2022-02-23      ┆ 3           ┆ 12-29          │
│ ANPOJ00000005 ┆ 2021-06-27      ┆ 1           ┆ 12-29          │
│ ANPOJ00000005 ┆ 2021-07-17      ┆ 2           ┆ 12-29          │
│ …             ┆ …               ┆ …           ┆ …              │
│ ANPOJ01307143 ┆ 2021-05-25      ┆ 1           ┆ 30-49          │
│ ANPOJ01307143 ┆ 2021-07-07      ┆ 2           ┆ 30-49          │
│ ANPOJ01307143 ┆ 2022-01-07      ┆ 3           ┆ 30-49          │
│ ANPOJ01307145 ┆ 2021-11-02      ┆ 1   

/tmp/ipykernel_15029/903785552.py:75: UserWarning: Sortedness of columns cannot be checked when 'by' groups provided
  predpis_df.join_asof(


In [ ]:
# === JUST FOR TESTING IF THE JOIN IS CORRECT ===
#  print(predpis_with_ocko_df.filter(pl.col("Pocet_davek_v_dobe_predpisu") > 0))

# test_id = "ANPOJ00000060"
# print(ocko_df.filter(pl.col("Id_pojistence") == test_id))
# print(predpis_df.filter(pl.col("Id_pojistence") == test_id))

# print(predpis_with_ocko_df.filter(pl.col("Id_pojistence") == test_id))

In [ ]:
vakcinace_grouped = (
    ocko_df.group_by(["Vekova_kohorta", "Datum_vakcinace", "Cislo_davky"])
    .agg(pl.len().alias("pocet_vakcinaci"))
    .sort(["Vekova_kohorta", "Datum_vakcinace", "Cislo_davky"])
)

print(vakcinace_grouped)

for vek_kohorta in vakcinace_grouped["Vekova_kohorta"].unique():
    for cislo_davky in vakcinace_grouped["Cislo_davky"].unique():
        data = vakcinace_grouped.filter(
            pl.col("Vekova_kohorta") == vek_kohorta,
            pl.col("Cislo_davky") == cislo_davky,
        )

        plt.figure(figsize=(10, 4))
        plt.plot(
            data["Datum_vakcinace"], data["pocet_vakcinaci"], marker="o", linestyle="-"
        )
        plt.title(
            f"Počet vakcinací: věková skupina {vek_kohorta}, při {cislo_davky}. dávce"
        )
        plt.xlabel("Datum vakcinace")
        plt.ylabel("Počet vakcinací")
        plt.xticks(rotation=45)
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(f"out/whole_period/cohort_{vek_kohorta}_{cislo_davky}.png")
        plt.close()

shape: (5, 4)
┌───────────────┬─────────────────┬─────────────┬────────────────┐
│ Id_pojistence ┆ Datum_vakcinace ┆ Cislo_davky ┆ Vekova_kohorta │
│ ---           ┆ ---             ┆ ---         ┆ ---            │
│ str           ┆ date            ┆ i64         ┆ str            │
╞═══════════════╪═════════════════╪═════════════╪════════════════╡
│ ANPOJ00000001 ┆ 2021-07-26      ┆ 1           ┆ 12-29          │
│ ANPOJ00000001 ┆ 2021-09-06      ┆ 2           ┆ 12-29          │
│ ANPOJ00000001 ┆ 2022-02-23      ┆ 3           ┆ 12-29          │
│ ANPOJ00000005 ┆ 2021-06-27      ┆ 1           ┆ 12-29          │
│ ANPOJ00000005 ┆ 2021-07-17      ┆ 2           ┆ 12-29          │
└───────────────┴─────────────────┴─────────────┴────────────────┘
shape: (16_338, 4)
┌────────────────┬─────────────────┬─────────────┬─────────────────┐
│ Vekova_kohorta ┆ Datum_vakcinace ┆ Cislo_davky ┆ pocet_vakcinaci │
│ ---            ┆ ---             ┆ ---         ┆ ---             │
│ str            ┆ date

In [25]:
rozhodna_data = (
    vakcinace_grouped.sort(
        ["Vekova_kohorta", "Cislo_davky", "pocet_vakcinaci", "Datum_vakcinace"],
        descending=[False, False, True, False],
    )
    .group_by(["Vekova_kohorta", "Cislo_davky"])
    .agg([pl.col("Datum_vakcinace").first().alias("Datum_rozhodne")])
)

print(rozhodna_data)

ocko_df_s_datem = ocko_df.join(
    rozhodna_data, on=["Vekova_kohorta", "Cislo_davky"], how="left"
).with_columns(
    (pl.col("Datum_vakcinace") - pl.col("Datum_rozhodne"))
    .dt.total_days()
    .alias("delta_dni")
)
# print(ocko_df_s_datem.head())


filtered_ocko_df = ocko_df_s_datem.filter(pl.col("delta_dni").abs() <= 30)

cohort_grouped_df = (
    filtered_ocko_df.group_by(["Vekova_kohorta", "Datum_vakcinace", "Cislo_davky"])
    .agg(pl.len().alias("pocet_vakcinaci"))
    .sort(["Vekova_kohorta", "Datum_vakcinace", "Cislo_davky"])
)

print(cohort_grouped_df)

for vek_kohorta in cohort_grouped_df["Vekova_kohorta"].unique():
    for cislo_davky in cohort_grouped_df["Cislo_davky"].unique():
        data = cohort_grouped_df.filter(
            pl.col("Vekova_kohorta") == vek_kohorta,
            pl.col("Cislo_davky") == cislo_davky,
        )

        plt.figure(figsize=(10, 4))
        plt.plot(
            data["Datum_vakcinace"], data["pocet_vakcinaci"], marker="o", linestyle="-"
        )
        plt.title(
            f"Počet vakcinací: věková skupina {vek_kohorta}, při {cislo_davky}. dávce"
        )
        plt.xlabel("Datum vakcinace")
        plt.ylabel("Počet vakcinací")
        plt.xticks(rotation=45)
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(f"out/cut_period/cohort_{vek_kohorta}_{cislo_davky}.png")
        plt.close()

shape: (53, 3)
┌────────────────┬─────────────┬────────────────┐
│ Vekova_kohorta ┆ Cislo_davky ┆ Datum_rozhodne │
│ ---            ┆ ---         ┆ ---            │
│ str            ┆ i64         ┆ date           │
╞════════════════╪═════════════╪════════════════╡
│ 30-49          ┆ 3           ┆ 2022-01-07     │
│ 50-59          ┆ 7           ┆ 2023-12-09     │
│ 80+            ┆ 5           ┆ 2023-10-11     │
│ <12            ┆ 2           ┆ 2022-01-11     │
│ 30-49          ┆ 4           ┆ 2022-10-07     │
│ …              ┆ …           ┆ …              │
│ 70-79          ┆ 4           ┆ 2022-10-14     │
│ 65-69          ┆ 2           ┆ 2021-06-07     │
│ 70-79          ┆ 3           ┆ 2021-12-02     │
│ 70-79          ┆ 6           ┆ 2023-10-11     │
│ <12            ┆ 1           ┆ 2021-12-21     │
└────────────────┴─────────────┴────────────────┘
shape: (2_183, 4)
┌────────────────┬─────────────────┬─────────────┬─────────────────┐
│ Vekova_kohorta ┆ Datum_vakcinace ┆ Cislo_davky